In [3]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph faiss-cpu

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/584.3 kB ? eta -:--:--
   --------------------------- ----------- 409.6/584.3 kB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 584.3/584.3 kB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/151.9 kB ? eta -:--:--
   ---------------------------------------- 151.9/151.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB ? eta 0:00:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.


## Model

In [116]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

import os

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in Python programming. Answer the user question \n
    in code you provide can be executed with all required imports and variables defined. \n
    Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:""",
        ),
        ("user", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Code output"""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")
    description = "Schema for code solutions to questions about Python."

assert os.path.exists("openai_key.txt")

with open("openai_key.txt", "r") as f:
    api_key = f.read().strip()

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key)
code_gen_chain = code_gen_prompt | llm.with_structured_output(code)

In [117]:
# Invoke the chain to get the solution
solution = code_gen_chain.invoke({"messages": [("user", "Build a RAG chain?")]})

In [119]:
# Access the structured output
print(solution.prefix)   # Description of the problem and approach
print(solution.imports)  # Required import statements
print(solution.code)     # The actual code block

To build a Retrieval-Augmented Generation (RAG) chain, we will use a combination of a retriever and a generator. The retriever will fetch relevant documents based on a query, and the generator will produce a response based on the retrieved documents. In this example, we will use a simple in-memory document store and a basic text generation model.
from transformers import pipeline
from typing import List, Tuple

# Sample documents for retrieval
documents = [
    "The sky is blue and beautiful.",
    "The sun is shining bright today.",
    "The quick brown fox jumps over the lazy dog.",
    "Python is a great programming language for data science."
]
def retrieve_documents(query: str, documents: List[str], top_k: int = 2) -> List[str]:
    # Simple keyword-based retrieval
    return [doc for doc in documents if query.lower() in doc.lower()][:top_k]


def generate_response(retrieved_docs: List[str]) -> str:
    generator = pipeline('text-generation', model='gpt2')
    combined_text = '\n'

## State

In [123]:
from typing import List, TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: str
    messages: List
    generation: str
    iterations: int

## Graph

In [126]:
from langchain_core.pydantic_v1 import BaseModel, Field

### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes


def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"messages": [("user", question)]}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"messages": [("user", question)]}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [128]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [130]:
question = "How do I build a RAG chain?"
app.invoke({"messages": [("user", question)], "iterations": 0})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
Based on the information retrieved:
Document 3: Guide to building web applications.
Document 4: Introduction to data science.
---NO CODE TEST FAILURES---
---DECISION: FINISH---


{'error': 'no',
 'messages': [('user', 'How do I build a RAG chain?'),
  ('assistant',
   'To build a Retrieval-Augmented Generation (RAG) chain, we need to combine a retriever and a generator. The retriever fetches relevant documents based on a query, and the generator produces a response using the retrieved documents. In this example, we will use a simple in-memory document store and a transformer-based model for generation. \n Imports: from transformers import pipeline\nfrom typing import List, Tuple\n\n# Sample documents for retrieval\ndocuments = [\n    "The sky is blue.",\n    "The sun is bright.",\n    "The grass is green.",\n    "The ocean is vast."\n] \n Code: def retrieve_documents(query: str, documents: List[str], top_k: int = 2) -> List[str]:\n    # Simple keyword-based retrieval\n    return [doc for doc in documents if query.lower() in doc.lower()][:top_k]\n\n\ndef generate_response(query: str, retrieved_docs: List[str]) -> str:\n    # Initialize a text generation pipeline